In [1]:
import pandas as pd
import psycopg2
import os, io
from dotenv import load_dotenv    

In [2]:
query = """
SELECT
    so.price_without_vat AS order_price_without_vat,
    so.price_with_vat AS order_price_with_vat,
    so.bill_country,
    so.setting_currency_id,
    so.created_at,
    so.shop_basket_id,
    so.doc_date,
    so.exchange_currency_rate,
    so.source_type AS source,

    sc.code AS currency_code,
    sc.currency_symbol,
    sc.price_round_system,


    sb.total_price_before_discount_with_vat AS basket_total_price_before_discount_with_vat,
    sb.total_price_with_vat AS basket_total_price_with_vat,
    sb.count_basket_items,
    sb.count_products AS basket_count_products,
    sb.basket_type,

    sbi.quantity AS item_quantity,
    sbi.item_type,
    sbi.unit_price_with_vat AS item_unit_price_with_vat,
    sbi.unit_price_without_vat AS item_unit_price_without_vat,
    sbi.total_discount_with_vat AS item_total_discount_with_vat,


    cp.id as product_id,
    cp.code AS product_code,
    cp.catalog_category_id,
    cp.catalog_brand_id,
    cp.name AS product_name,
    cp.status AS product_status,
    cp.reviews_count,
    cp.reviews_average_score_price,
    cp.reviews_average_score_quality,
    cp.reviews_average_score_properties,
    cp.reviews_average_score_overall,
    cp.reviews_average_score,
    cp.is_in_stock,
    cp.is_ended,
    cp.is_new,
    cp.is_boosted,
    cp.purchase_price AS product_purchase_price,
    cp.eshop_stock_count,
    cp.is_fifo,
    cp.name_parameterize AS product_name_parameterize,

    cc.name AS category,
    cc.tree_path,
    cc.name_parameterize AS category_name_parameterized,
    cc.status AS category_status,
    cc.catalog_segment_id,
    cc.ancestor_ids AS categories_ancestor_ids,
    cc.descendant_ids AS categories_descendant_ids,
    cc.full_name_path AS category_full_name_path,
    cc.default_warranty_period,

    cb.name AS brand_name,
    cb.name_parameterize AS brand_parameterized,

    cs.name AS segment_name,
    cs.name_parameterize AS segment_parameterezied,
    cs.status AS segment_status


FROM shop_orders so
LEFT JOIN setting_currencies sc ON so.setting_currency_id = sc.id
INNER JOIN shop_baskets sb ON sb.id = so.shop_basket_id
LEFT JOIN shop_basket_items sbi ON sbi.id = so.shop_basket_id
INNER JOIN catalog_products cp ON cp.id = sbi.catalog_product_id
LEFT JOIN catalog_categories cc ON cp.catalog_category_id = cc.id
LEFT JOIN catalog_brands cb ON cp.catalog_brand_id = cb.id
LEFT JOIN catalog_segments cs ON cs.id = cp.catalog_segment_id
"""

In [3]:
query_products = """
SELECT id AS variant_id,
       catalog_category_id AS category_id,
       catalog_brand_id AS brand_id,
       status,
       is_new,
       reviews_count,
       reviews_average_score_price,
       reviews_average_score_quality,
       reviews_average_score_properties,
       reviews_average_score_overall,
       in_sets_count,
       is_in_stock,
       is_returnable,
       purchase_price,
       eshop_stock_count,
       country_of_origin_code,
       name
       
       FROM catalog_products
"""

In [4]:
load_dotenv()

True

In [5]:
def create_db_conn():
    try:
        conn = psycopg2.connect(host=os.getenv('DB_HOST'), database=os.getenv('DB_NAME'),
                                             user=os.getenv('DB_USER'), password=os.getenv('DB_PASSWORD'),
                                         port=int(os.getenv('DB_PORT')))
        return conn
    except psycopg2.DatabaseError as e:
        print(f'database connection {e}')
        return None
    except Exception as e:
        print(f'unknown error {e}')
        return None


def read_sql_iostream(query: str, block_mergejoin=False, block_hashjoin=False, block_seqscan=False) -> pd.DataFrame:
    """
    More effective way of loading content of database table to dataframe using io stream - StringIO.
    :param str query: Query select for accessing data in table.
    :param con: Connection to concrete database.
    :return pd.Dataframe: Output dataframe loaded from database.
    """
    try:
        con = create_db_conn()
        cur = con.cursor()
        copy_sql = f"COPY ({query.strip().rstrip(';')}) TO STDOUT WITH CSV HEADER"
        store = io.StringIO()
        cur.copy_expert(copy_sql, store)
        store.seek(0)
        df = pd.read_csv(store, na_values=['NULL', 'NaN', 'nan', 'null', ''], keep_default_na=False)
    except Exception as e:
        raise e
    finally:
        try:
            cur.close()
            con.close()
        except Exception as e:
            print(f'error- {e}')
            pass
    return df

In [6]:
# Check whether the data folder exists or not
if not os.path.exists('./data'):
    os.makedirs('./data')

In [7]:
orders = read_sql_iostream(query)

In [8]:
orders.columns

Index(['order_price_without_vat', 'order_price_with_vat', 'bill_country',
       'setting_currency_id', 'created_at', 'shop_basket_id', 'doc_date',
       'exchange_currency_rate', 'source', 'currency_code', 'currency_symbol',
       'price_round_system', 'basket_total_price_before_discount_with_vat',
       'basket_total_price_with_vat', 'count_basket_items',
       'basket_count_products', 'basket_type', 'item_quantity', 'item_type',
       'item_unit_price_with_vat', 'item_unit_price_without_vat',
       'item_total_discount_with_vat', 'product_id', 'product_code',
       'catalog_category_id', 'catalog_brand_id', 'product_name',
       'product_status', 'reviews_count', 'reviews_average_score_price',
       'reviews_average_score_quality', 'reviews_average_score_properties',
       'reviews_average_score_overall', 'reviews_average_score', 'is_in_stock',
       'is_ended', 'is_new', 'is_boosted', 'product_purchase_price',
       'eshop_stock_count', 'is_fifo', 'product_name_paramete

In [9]:
orders

,order_price_without_vat,order_price_with_vat,bill_country,setting_currency_id,created_at,shop_basket_id,doc_date,exchange_currency_rate,source,currency_code,...,catalog_segment_id,categories_ancestor_ids,categories_descendant_ids,category_full_name_path,default_warranty_period,brand_name,brand_parameterized,segment_name,segment_parameterezied,segment_status
0,32724.41000,41560.00,HU,10,2020-04-26 19:25:42.059702,812554,2020-04-26,359.9200,eshop,HUF,...,13.0,"{1,2014}",{},"{""Muziker Merch"",""Merch - Miscellaneous""}",24.0,Muziker,muziker,General,general,inactive
1,28.52000,34.79,IT,6,2020-04-26 19:26:32.223119,1182415,2020-04-26,1.0000,eshop,EUR,...,13.0,"{1,2014}",{},"{""Muziker Merch"",""Merch - Miscellaneous""}",24.0,Muziker,muziker,General,general,inactive
2,929.41000,1106.00,RO,13,2022-02-03 13:56:28.395746,4358127,2022-02-03,4.9463,eshop,RON,...,4.0,"{1327,1572,1580}",{},"{Boats,""Paint / Care / Tools"",""Cleaning / Care...",24.0,Star Brite,star-brite,Boats,boats,active
3,125.33000,150.39,SK,6,2020-04-26 19:24:18.330764,1173713,2020-04-26,1.0000,eshop,EUR,...,1.0,"{3,4,5}",{},"{Music,Guitars,""Electric Guitars"",ST-Type}",24.0,ABX,abx,Music,music,active
4,20.58000,24.70,FR,6,2020-04-26 19:17:46.399952,1182405,2020-04-26,1.0000,eshop,EUR,...,1.0,"{3,4,5}",{},"{Music,Guitars,""Electric Guitars"",""Hard & Heavy""}",24.0,Jackson,jackson,Music,music,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156066,626.49000,751.59,SK,6,2022-03-12 19:06:28.633632,2584127,2022-03-12,1.0000,eshop,EUR,...,1.0,"{3,607,683}",{},"{Music,""Studio / PA"",""Complete PA-Sets"",""Sets ...",24.0,Vonyx,vonyx,Music,music,active
1156067,27346.46000,34730.00,HU,10,2022-03-12 18:44:06.224012,4524139,2022-03-12,380.9200,eshop,HUF,...,3.0,"{1182,1267}",{},"{Bikes,""Cycling accessories"",Bike-gloves}",24.0,Oakley,oakley,Bike,bike,active
1156068,22899.95993,29080.00,HU,10,2022-02-28 11:56:34.143569,4356245,2022-02-28,365.2800,eshop,HUF,...,7.0,"{1883,1253,1260}",{},"{Moto,Garage,""Motorcycle Oils, Filters, Lubric...",24.0,Yamalube,yamalube,Moto,moto,active
1156069,21.24000,25.49,GB,7,2022-03-15 09:26:46.337213,4534075,2022-03-15,0.8392,eshop,GBP,...,1.0,"{3,607,667}",{},"{Music,""Studio / PA"",Loudspeakers,""Active Loud...",24.0,Mackie,mackie,Music,music,active


In [10]:
products = read_sql_iostream(query_products)
products

,variant_id,category_id,brand_id,status,is_new,reviews_count,reviews_average_score_price,reviews_average_score_quality,reviews_average_score_properties,reviews_average_score_overall,in_sets_count,is_in_stock,is_returnable,purchase_price,eshop_stock_count,country_of_origin_code,name
0,45528,2759.0,586.0,active,f,0,0.0,0.0,0.0,0.0,0,f,t,2317.000000,0.0,DE,Saphir 41/120/IV/11/5 Green
1,293638,1138.0,872.0,active,f,1,0.0,0.0,0.0,0.0,0,t,t,28.280000,2.0,CN,Roshe G Junior Golf Shoes Lime US 7Y
2,211291,2117.0,4959.0,ended,f,0,0.0,0.0,0.0,0.0,0,f,t,1.230000,0.0,CN,Flora 21 Pink
3,327605,2042.0,2088.0,active,f,0,0.0,0.0,0.0,0.0,0,t,f,15.915556,9.0,DE,Guitar Shop (LP)
4,243315,2142.0,6525.0,active,f,2,0.0,0.0,0.0,0.0,0,t,t,7.597500,17.0,PT,PC951 9-Speed Chain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351048,75502,1131.0,872.0,ended,f,0,0.0,0.0,0.0,0.0,0,f,t,NaN,0.0,VN,Boys Therma Top Hz University Red/White S
351049,78948,1131.0,921.0,ended,f,0,0.0,0.0,0.0,0.0,0,f,t,NaN,0.0,CN,Dwayne Tour Insula Mens Sweater Kings Blue 3XL
351050,81765,1131.0,866.0,ended,f,0,0.0,0.0,0.0,0.0,0,f,t,NaN,0.0,CN,1/4 Zip Blocked Mens Sweater Caviar XL
351051,162843,60.0,616.0,archived,f,0,0.0,0.0,0.0,0.0,0,f,t,17.080000,0.0,CN,Soprano Ukulele Set (B-Stock) #923506


In [11]:
orders.to_csv('data/data.csv', index=False)
products.to_csv('data/products.csv', index=False)